In [1]:
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import pandas as pd

In [2]:
class DatasetDownloader:
    def __init__(self, base_url, download_folder):
        self.base_url = base_url
        self.download_folder = Path(download_folder)
        # Asegúrate de que el directorio de descarga exista
        self.download_folder.mkdir(parents=True, exist_ok=True)

    def scrape_siniestros_urls(self):
        response = requests.get(self.base_url)
        soup = BeautifulSoup(response.content, 'html.parser')
        links = soup.find_all('a')
        
        file_urls = {}
        for link in links:
            href = link.get('href')
            if href and href.endswith(('.zip', '.xlsx', '.csv', '.gz', '.geojson','.pdf')):
                file_name = href.split('/')[-1]
                file_urls[file_name] = href
        return file_urls

    def download_file(self, url, save_path):
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(save_path, 'wb') as f:
                f.write(response.content)
            print(f"Archivo guardado: {save_path}")
        else:
            print(f"Error al descargar el archivo: {url}")

    def download_datasets(self):
        file_urls_dict = self.scrape_siniestros_urls()
        for file_name, url in file_urls_dict.items():
            save_path = self.download_folder / file_name
            self.download_file(url, save_path)

base_url = 'https://data.buenosaires.gob.ar/dataset/victimas-siniestros-viales'
download_folder = 'datasets/raw'
downloader = DatasetDownloader(base_url, download_folder)
downloader.download_datasets()

Archivo guardado: datasets\raw\lesiones.xlsx
Archivo guardado: datasets\raw\homicidios.xlsx
Archivo guardado: datasets\raw\NOTAS_LESIONES_SINIESTRO_VIAL.pdf
Archivo guardado: datasets\raw\NOTAS_HOMICIDIOS_SINIESTRO_VIAL.pdf
